In [4]:
# environment: sceneGraphs_groundTruth_Replica

In [42]:
from plyfile import *
import numpy as np
import os
import open3d as o3d
import json

# Get the files with the segmentation

In [17]:
# Code from: https://github.com/facebookresearch/Replica-Dataset/issues/17

path_in_base = '/local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/habitat/' #TODO TOSET
name = "mesh_semantic.ply"
path_in = os.path.join(path_in_base, name)

print("Reading input...")
file_in = PlyData.read(path_in)
vertices_in = file_in.elements[0]
faces_in = file_in.elements[1]

print("Filtering data...")
objects = {}
for f in faces_in:
     object_id = f[1]
     if not object_id in objects:
         objects[object_id] = []
     objects[object_id].append((f[0],))

print("Writing data...")
segmentation_dir = os.path.join(path_in_base, "Segmentation/")
os.makedirs(segmentation_dir, exist_ok=True)
for object_id, faces in objects.items():
    path_out = segmentation_dir + name + f"_{object_id}.ply"
    faces_out = PlyElement.describe(np.array(faces, dtype=[('vertex_indices', 'O')]), 'face')
    PlyData([vertices_in, faces_out]).write(path_out)

Reading input...
Filtering data...
Writing data...


In [52]:
#
# Visualise an object
#

# Specify the path to one of the output PLY files
path_to_output_ply = '/local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/habitat/Segmentation/mesh_semantic.ply_47.ply' #TODO TOSET

# Load the PLY file
mesh = o3d.io.read_triangle_mesh(path_to_output_ply)

# Check if the mesh was loaded correctly
if not mesh.has_triangles():
    raise ValueError(f"Failed to load mesh from {path_to_output_ply}")

# Compute vertex normals (if they are not already present)
mesh.compute_vertex_normals()

# Visualize the mesh
o3d.visualization.draw_geometries([mesh])

# Get useful information

In [59]:
# Get a list list_file_paths containing the paths of all the meshes (one mesh represents one object). Ordered following the numbering of the meshes

all_items = os.listdir(segmentation_dir)
list_file_paths = [os.path.join(segmentation_dir, item) for item in all_items if os.path.isfile(os.path.join(segmentation_dir, item))]
list_file_paths.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

print(len(list_file_paths))


233


In [60]:
# Create list_pointsPointClouds. Each element is a point cloud, there is a correspondance with list_file_paths

np.random.seed(42)

list_pointsPointClouds = [] # list_pointsPointClouds[i] contains the points related to list_file_paths[i] (i.e., mesh_semantic.ply_i.ply). i represents the obj_id

# for i in range(len(list_file_paths)): #TODO: sistemare
for i in range(6):
    path_to_mesh_ply = list_file_paths[i]
    mesh = o3d.io.read_triangle_mesh(path_to_mesh_ply)
    mesh.compute_vertex_normals()
    # Sample points from the mesh
    point_cloud = mesh.sample_points_uniformly(number_of_points=1000)  #TODO TOSET: adjust the number of points as needed
    list_pointsPointClouds.append(point_cloud)

RPly: Aborted by user
RPly: Aborted by user
RPly: Aborted by user


6


RPly: Aborted by user


In [65]:
# Create list_info with information for each object. REFER TO THE id WHEN SEARCHING FOR SOMETHING

# TODO: an idea could be to add a different color for each obj_id
# TODO: get the segmentation point cloud. But in that case we should choose the colors to give for each obj_id. They may also be random

name_semantic = "info_semantic.json"
path_info_semantic = os.path.join(path_in_base, name_semantic)

with open(path_info_semantic, 'r') as file:
    data = json.load(file)

objects = data.get('objects', [])
list_info = [] # remark that it will not be ordered

length = len(objects)

for obj in objects:
    obj_id = obj.get('id')
    class_name = obj.get('class_name')
    center = obj.get('oriented_bbox', {}).get('abb', {}).get('center', [])
    list_info.append([obj_id, class_name, center])

print(len(list_info))

226


Be aware that we may have that `len(list_file_paths) != len(list_info)`, but I don't really know why. Some ids seem to be missing in the `.json` file

# Get the scene graph